## Setup Street Fighter

In [ ]:
# Import gym-retro to load ROM
import retro
# Import time to set FPS
import time

In [ ]:
!python -m retro.import roms

In [ ]:
# list all possible games
retro.data.list_games()

In [ ]:
# load game (can only open one at a time)
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [ ]:
env.close()

In [ ]:
# (200, 256, 3) image
env.observation_space

In [ ]:
# 12 actions and all combinations of them
env.action_space

In [ ]:
# simple game loop
obs = env.reset()
done = False
for game in range(2):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)
        time.sleep(1/30)

## Setup Custom environment

- Preprocess
    - grayscale
    - frame delta
    - resize image smaller
- Filter actions to less combinations
- Change reward function
    - Set to score

In [ ]:
# Import wrapper base class
from gym import Env
# import space shapes  for the env
from gym.spaces import MultiBinary, Box
# For calculating change from one frame to the next
import numpy as np
# For grayscaling
import cv2 as cv
# For plotting images
from matplotlib import pyplot as plt

In [ ]:
class StreetFighter(Env):
    def __init__(self) -> None:
        super().__init__()
        
        # Specify action and observation spaces
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8) # shape=(200, 256, 3)
        self.action_space = MultiBinary(12)
        
        # start up game
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis',
                               use_restricted_actions=retro.Actions.FILTERED)
    
    def step(self, action):
        # take step
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
        # frame delta
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        
        # new reward function
        reward = info['score'] - self.score
        self.score = info['score']
        
        return frame_delta, reward, done, info
        
    def render(self, *args, **kwargs):
        self.game.render()
    
    def reset(self):
        obs = self.game.reset()
        
        # need the first frame to calculate delta for first frame
        obs = self.preprocess(obs)
        self.previous_frame = obs
        
        # same for score
        self.score = 0
        
        return obs
    
    def preprocess(self, observation):
        obs = cv.cvtColor(observation, cv.COLOR_RGB2GRAY)
        obs = cv.resize(obs, (84, 84), cv.INTER_CUBIC)
        obs = np.expand_dims(obs, -1)
        return obs
    
    def close(self):
        self.game.close()
        return super().close()

In [ ]:
env.close()
env.game.close()

In [ ]:
env = StreetFighter()

In [ ]:
# simple game loop
obs = env.reset()
done = False
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        if reward > 0:
            print(reward)
        time.sleep(1/60)